In [ ]:
%matplotlib widget

: 

In [14]:
import os    
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from wordcloud import WordCloud
import numpy as np
from collections import Counter
from konlpy.tag import Okt
from matplotlib import pyplot as plt
import re
import SimpleITK as sitk

with open('.env', 'r') as file:
	font_path = file.readline() # 실제 폰트 저장소 위치

with open('stop_words.txt', 'r') as file:
    stop_words = file.readline().split('|')

browser = webdriver.Chrome(ChromeDriverManager().install()) 
# browser 변수에  chromeDriveManager 최신 버전 (자동 업데이트) 할당
browser.implicitly_wait(10) # 브라우저 로딩까지 10초간 대기

wantedUrl = 'https://www.wanted.co.kr/wdlist/518?country=kr&job_sort=company.response_rate_order&years=-1&locations=all' # id parameter를 가져올 base url

browser.get(f'{wantedUrl}') # webdriver_manager가 url에 접속하게 함

html = browser.page_source # 페이지에서 html을 추출
soup = bs(html, 'html.parser') # bs4 모듈로 html 파싱

urls = [] # 빈 리스트 생성

length = len(str(soup.select('div .List_List_container__JnQMS > ul > li > div > a')).split('<a aria-label')) - 1 # 총 공고 개수 가져오기

for i in range(length):
    param = soup.select('div .List_List_container__JnQMS > ul > li > div > a')[i]['href'] # 각 id parameter 가져오기
    urls.append(f'https://www.wanted.co.kr{param}') # urls 리스트에 각 id parameter를 추가한 URL 추가

ceritifies = ''

for i in range(length):
    try:
        browser.get(urls[i])

        soup = str(bs(browser.page_source, 'html.parser').select('.JobDescription_JobDescription__VWfcb span')) # 게시글 내용 가져옴

        ceritify = soup.split('<span>')[3].replace('</span>', '').replace('<br/>', '') # 자격요건
        # then = soup.split('<span>')[4].replace('</span>', '').replace('<br/>', '').split('•') #우대사항

        ceritifies += ceritify
    except IndexError:
        print(soup)
        ceritify = ''
        ceritifies += ceritify
        

# count = Counter(Okt().morphs(phrase=ceritifies, stem=True))

# for leng in count :
#      if(len(leng) <= 1): del count[leng]

c = Okt().morphs(phrase=ceritifies, stem=True)

for leng in c :
     for stop in stop_words:
          if stop in leng :
               c.remove(leng)
               break

count = Counter(c)

wc = WordCloud(
     font_path=font_path, # 서체 지정
       width= 400, 
       height=400, 
       scale=2.0, 
       max_font_size=100, 
       background_color="white", 
       collocations=False
     ).generate_from_frequencies(count)


wc.to_file('test.png') # 파일로 다운로드

plt.figure(figsize=(15, 15))
plt.imshow(wc)
